In [1]:
import ijson
import json

In [2]:
# the original json expanded from the .7z file is missing a closing ] for it to be parsed by the json parser
with open('dblpv13.json', 'rb') as infile, open('dblp.json', 'wb') as outfile:
    string = infile.read().rstrip()
    outfile.write(string)
    outfile.write(b']\n')

In [17]:
with open('dblp.json', 'rb') as infile, open('dblp2010.json', 'wb') as outfile:
    pubs = ijson.items(infile, 'item')
    outfile.write(b'[')
    count = 0
    for pub in pubs:
        if 'year' not in pub.keys() or 'authors' not in pub.keys():
            continue
        if 'n_citation' not in pub.keys() or int(pub['n_citation']) == 0:
            continue
        if 'keywords' not in pub.keys() or len(pub['keywords']) == 0:
            continue
        if 'url' not in pub.keys() or len(pub['url']) == 0:
            continue
        if int(pub['year']) >= 2010:
            outfile.write(json.dumps(pub).encode('utf-8'))
            outfile.write(b',\n')
            count += 1
    outfile.seek(-2, 1)
    outfile.write(b']\n')
    print(count)

1161982


5354309 records total  
4620772 records since 2000(2001)  
3281786 records since 2010  
1765168 records since 2010 with nonzero citations  
1161982 records since 2010 with nonzero citations, urls, keywords  
1145118 records since 2010 with nonzero citations, urls, keywords, fields of study  

In [15]:
with open('dblp2010.json', 'rb') as infile,\
    open('dblp2010_p2a.json', 'wb') as p2aout,\
    open('dblp2010_a2p.json', 'wb') as a2pout:

    pubs = ijson.items(infile, 'item')
    p2aout.write(b'[')
    a2pout.write(b'[')
    a2p = {}
    count = 0
    noid_auth_count = 0
    noid_venue_count = 0
    for pub in pubs:
        if 'venue' not in pub.keys():
            continue
        if 'fos' not in pub.keys() or len(pub['fos']) == 0:
            continue

        new_pub = {}
        new_pub['_id'] = pub['_id']
        new_pub['title'] = pub['title']
        new_pub['authors'] = []
        should_write = True
        for author in pub['authors']:
            if '_id' not in author or 'name' not in author:
                continue
            # if author['name'] == 'Jeffrey Heer':
            #     should_write = True
            new_pub['authors'].append({'_id':author['_id'],'name':author['name']})
        new_pub['year'] = pub['year']
        if '_id' not in pub['venue']:
            noid_venue_count += 1
            continue
        venue_name = ''
        if 'raw' not in pub['venue']:
            if 'name' not in pub['venue']:
                if 'name_d' not in pub['venue']:
                    venue_name = 'N/A'
                else:
                    venue_name = pub['venue']['name_d']
            else:
                venue_name = pub['venue']['name']
        else:
            venue_name = pub['venue']['raw']
        new_pub['venue'] = {'_id':pub['venue']['_id'],'name':venue_name}
        new_pub['keywords'] = pub['keywords']
        new_pub['fos'] = pub['fos']
        new_pub['n_citation'] = pub['n_citation']
        new_pub['url'] = pub['url'][0]
        if should_write:
            p2aout.write(json.dumps(new_pub).encode('utf-8'))
            p2aout.write(b',\n')
            count += 1
        else:
            continue

        for author in pub['authors']:
            if '_id' not in author.keys():
                noid_auth_count += 1
                continue
            if 'name' not in author:
                continue
            if author['_id'] not in a2p:
                a2p[author['_id']] = {'name':author['name'], 'papers':[]}
            a2p[author['_id']]['papers'].append(pub['_id'])
            a2p[author['_id']]['email'] = author['email'] if 'email' in author else ''
            if 'org' in author:
                a2p[author['_id']]['org'] = author['org']
            elif 'orgs' in author and len(author['orgs']) > 0:
                a2p[author['_id']]['org'] = author['orgs'][0]

    p2aout.seek(-2, 1)
    p2aout.write(b']\n')
    
    for key, value in a2p.items():
        a2pout.write(json.dumps({key: value}).encode('utf-8'))
        a2pout.write(b',\n')
    a2pout.seek(-2, 1)
    a2pout.write(b']\n')

    print(count)
    print(noid_auth_count)
    print(noid_venue_count)

747165
94292
397953


In [46]:
with open('dblp2010_a2p.json', 'wb') as a2pout:
    a2pout.write(b'[')
    for key, value in a2p.items():
        a2pout.write(json.dumps({key: value}).encode('utf-8'))
        a2pout.write(b',\n')
    a2pout.seek(-2, 1)
    a2pout.write(b']\n')

357769 authors no author id (got filtered away)  
397953 venue no id  
747165 left with all the data we need for the vis

In [16]:
with open('dblp2010_a2p.json', 'rb') as a2pin, open('dblp2010_a2p2.json', 'wb') as a2pout:
    a2pout.write(b'[')
    a2p = json.load(a2pin)
    for entry in a2p:
        for key in entry.keys():
            newentry = {"_id" : key}
            newentry.update(entry[key])
            a2pout.write(json.dumps(newentry).encode('utf-8'))
            a2pout.write(b',\n')
    a2pout.seek(-2, 1)
    a2pout.write(b']\n')